# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
#load and read csv and creat a df 
city_weather_data = "output_vacation/city_weather.csv"

city_weather_data = pd.read_csv(city_weather_data)

# Combine the data into a single dataset
city_weather = pd.DataFrame(city_weather_data)
city_weather


,Unnamed: 0,City,Country,Date,Max Temp,Humidity,Cloudiness,Wind Speed,Latitude,Longitude
0,103,Torbay,CA,1603627200,49.98,99,95,15.79,47.6666,-52.7314
1,245,Mnogovershinnyy,RU,1603627200,36.10,99,100,13.20,53.9353,139.9242
2,522,Kiruna,SE,1603627200,28.17,99,91,3.22,67.8557,20.2251
3,171,Gari,RU,1603627200,31.35,99,95,2.10,59.4307,62.3497
4,307,Vuktyl,RU,1603627200,26.94,98,100,7.61,63.8567,57.3094
...,...,...,...,...,...,...,...,...,...,...
536,478,Harsīn,IR,1603627200,79.97,10,0,7.07,34.2721,47.5861
537,239,Djibo,BF,1603627200,98.10,10,0,14.83,14.1022,-1.6306
538,262,Filingué,NE,1603627200,97.95,10,59,10.89,14.3521,3.3168
539,513,Senanga,ZM,1603627200,102.52,9,0,10.07,-16.1167,23.2667


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#Configure gmaps
gmaps.configure(api_key=g_key)

In [6]:
#Store lat and long as locations and hum as weight
locations = city_weather[["Latitude", "Longitude"]].astype(float)
hum = city_weather["Humidity"].astype(float)

In [19]:
#Add heatmap layer to map
fig = gmaps.figure(center=(50.0, -100.0), zoom_level=3)

heatmap_layer = gmaps.heatmap_layer(locations,
                                    weights=hum,
                                    opacity=1,
                                    dissipating=False,
                                    max_intensity=100,
                                    point_radius = 1)

fig.add_layer(heatmap_layer)

#saving figure
#plt.savefig('output_vacation/heatmap_hum.png')

fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [47]:
#Narrow down the weather df to find ideal weather 

#Temp: Max temperature lower than 85 degrees but higher than 70.
temp_1 = city_weather.loc[city_weather["Max Temp"]<85,:]
ideal_temp = temp_1.loc[city_weather["Max Temp"]>70,:]

# =<20% cloudiness
cloud = ideal_temp.loc[ideal_temp["Cloudiness"]<=20,:]

# =<30% humidity 
hum = cloud.loc[cloud["Humidity"]<=50,:]

#Wind speed less than 10 mph.
ideal_cities = hum.loc[hum["Wind Speed"]<=10,:]

ideal_cities


,Unnamed: 0,City,Country,Date,Max Temp,Humidity,Cloudiness,Wind Speed,Latitude,Longitude
480,365,Samsun Province,TR,1603627200,79.14,44,0,3.62,41.2500,36.3333
481,407,Qax,AZ,1603627200,73.74,44,0,3.38,41.4225,46.9242
483,439,Ginda,IN,1603627200,70.86,42,10,2.91,30.6945,78.4932
487,377,Awjilah,LY,1603627200,79.90,38,0,7.94,29.1081,21.2869
495,442,Kumluca,TR,1603627200,83.44,33,0,6.98,36.3703,30.2869
522,473,Sīrjān,IR,1603627200,74.08,16,0,7.47,29.4520,55.6814
526,305,Fasa,IR,1603627200,79.07,13,0,1.10,28.9383,53.6482
536,478,Harsīn,IR,1603627200,79.97,10,0,7.07,34.2721,47.5861


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [48]:
#Store into variable named hotel_df.
#Add a "Hotel Name" column to the DF.
hotel_df = pd.DataFrame()
hotel = pd.DataFrame(ideal_cities)
hotel_df['City'] = hotel['City']
hotel_df['Country'] = hotel['Country']
hotel_df['Lat'] = hotel['Latitude']
hotel_df['Lng'] = hotel['Longitude']

hotel_df['Hotel Name'] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
480,Samsun Province,TR,41.2500,36.3333,
481,Qax,AZ,41.4225,46.9242,
483,Ginda,IN,30.6945,78.4932,
487,Awjilah,LY,29.1081,21.2869,
495,Kumluca,TR,36.3703,30.2869,
522,Sīrjān,IR,29.4520,55.6814,
526,Fasa,IR,28.9383,53.6482,
536,Harsīn,IR,34.2721,47.5861,


In [49]:
#go through each city row in df
for index, row in hotel_df.iterrows():

    # get city info from df
    latitude = row['Lat']
    longitude = row['Lng']
    city = row['City']
    country = row['Country']

    # url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(f"https://maps.googleapis.com/maps/api/place/textsearch/json?location={latitude},{longitude}&radius=5000&type=hotel&key={g_key}").json()
    
    # extract results
    results = response['results']
    
    try:
        hotelname = response['results'][7]['name']
        print(f"Closest hotel to {city} at {latitude} , {longitude} is {hotelname}.")
        
        hotel_df.loc[index, "Hotel Name"] = hotelname
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("~~~~~~~~~~~~~~~~~~~~~~~~")

Retrieving Results for Index 480: Samsun Province.
Closest hotel to Samsun Province at 41.25 , 36.3333 is Samsun Hotel Residence.
~~~~~~~~~~~~~~~~~~~~~~~~
Retrieving Results for Index 481: Qax.
Closest hotel to Qax at 41.4225 , 46.9242 is AZSTUDIO.
~~~~~~~~~~~~~~~~~~~~~~~~
Retrieving Results for Index 483: Ginda.
Closest hotel to Ginda at 30.6945 , 78.4932 is Government Inter College, Manpur.
~~~~~~~~~~~~~~~~~~~~~~~~
Retrieving Results for Index 487: Awjilah.
Closest hotel to Awjilah at 29.1081 , 21.2869 is Bank of Commerce & Development, Awjilah.
~~~~~~~~~~~~~~~~~~~~~~~~
Retrieving Results for Index 495: Kumluca.
Closest hotel to Kumluca at 36.3703 , 30.2869 is Elit Cafe Bar.
~~~~~~~~~~~~~~~~~~~~~~~~
Retrieving Results for Index 522: Sīrjān.
Closest hotel to Sīrjān at 29.451999999999998 , 55.6814 is شرکت الکتروکام گستر جنوب.
~~~~~~~~~~~~~~~~~~~~~~~~
Retrieving Results for Index 526: Fasa.
Closest hotel to Fasa at 28.9383 , 53.6482 is Melli Bank.
~~~~~~~~~~~~~~~~~~~~~~~~
Retrieving Res

In [50]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
480,Samsun Province,TR,41.2500,36.3333,Samsun Hotel Residence
481,Qax,AZ,41.4225,46.9242,AZSTUDIO
483,Ginda,IN,30.6945,78.4932,"Government Inter College, Manpur"
487,Awjilah,LY,29.1081,21.2869,"Bank of Commerce & Development, Awjilah"
495,Kumluca,TR,36.3703,30.2869,Elit Cafe Bar
522,Sīrjān,IR,29.4520,55.6814,شرکت الکتروکام گستر جنوب
526,Fasa,IR,28.9383,53.6482,Melli Bank
536,Harsīn,IR,34.2721,47.5861,دفتر پیشخوان دولت وارتباطات مبین72301236


In [51]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations2 = hotel_df[["Lat", "Lng"]]

In [52]:
# Add marker layer ontop of heat map

fig = gmaps.figure(center=(50.0, -100.0), zoom_level=1)

heatmap_layer = gmaps.heatmap_layer(locations,
                                    weights=hum,
                                    opacity=1,
                                    dissipating=False,
                                    max_intensity=100,
                                    point_radius = 1)

marker_layer = gmaps.marker_layer(locations2,
                                  weights=hum,
                                  point_radius = 1,
                                  hover_text='', 
                                  label='', 
                                  info_box_content=hotel_info)

fig.add_layer(heatmap_layer)
fig.add_layer(marker_layer)

plt.savefig("vacation_map_hotels.png")

fig


ValueError: weights must be of the same length as locations or None